In [6]:
import os
os.environ["TAVILY_API_KEY"] = "TAVILY-API-KEY"

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [8]:
searchtool = TavilySearchResults(max_results=3)


In [27]:
searchtool.invoke({'query': 'Who won the UEFA European Football Championship in 2024?'})

[{'url': 'https://www.lemonde.fr/en/sport/euro-2024/results/',
  'content': '... UEFA European Football Championship pages on « Le Monde » ... Euro 2024: Spain beats England 2-1 and wins record fourth European Championship title.'},
 {'url': 'https://en.wikipedia.org/wiki/UEFA_Euro_2024_final',
  'content': 'In front of a crowd of 65,600, Spain won the match 2–1 for their record-breaking fourth UEFA European Championship title (after 1964, 2008 and 2012), surpassing Germany as the sole record-winners of the competition. It was also their third title in the last five editions of the tournament. [...] Details[edit]\n14\xa0July\xa02024\n21:00 CEST\nSpain\xa0  2–1 \xa0England\nWilliams  47\'\nOyarzabal  86\'\n    Report\nPalmer  73\'\nOlympiastadion, Berlin\nAttendance: 65,600[3]\nReferee: François Letexier (France)\nSpain[4]\nEngland[4]\nGK  23  Unai Simón\nRB  2   Dani Carvajal\nCB  3   Robin Le Normand        \xa083\'\nCB  14  Aymeric Laporte\nLB  24  Marc Cucurella\nCM  16  Rodri      

In [28]:
tools = [searchtool]


In [29]:
from langchain_core.prompts import ChatPromptTemplate


In [30]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Make sure to invoke tavily_search_results_json."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [31]:
from langchain.agents import AgentExecutor, create_tool_calling_agent


In [32]:
agent = create_tool_calling_agent(model, tools, prompt)


In [15]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, stream_runnable=False)

In [16]:
agent_executor.invoke({"input": "Who won the UEFA European Football Championship in 2024?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'UEFA European Football Championship 2024 winner'}`


[{'url': 'https://www.cnn.com/2024/07/14/sport/spain-england-euro-2024-final-spt-intl/index.html', 'content': 'Alvaro Morata lifts the Henri Delaunay trophy after Spain defeats England in the Euro 2024 final.\nMichael Regan/UEFA/Getty Images\nCNN \xa0—\xa0\nSpain won a record-breaking fourth European Championship, defeating England 2-1 following a drama-filled second half in the Euro 2024 final on Sunday in Berlin. [...] Ad Feedback\nSports / Football\nSpain wins Euro 2024, defeating England 2-1 in a dramatic final to claim record fourth European Championship\nBy Ben Morse, CNN\n5 minute read\nUpdated 5:44 PM EDT, Sun July 14, 2024\n\nLink Copied!\nFollow:\nUK See your latest updates [...] Spain wins Euro 2024, defeating England 2-1 in a dramatic final to claim record fourth European Championship | CNN'}, {'url': 'https://en.wikipedia.org

{'input': 'Who won the UEFA European Football Championship in 2024?',
 'output': "Spain won the UEFA European Football Championship in 2024 by defeating England 2-1 in the final match. This victory marked Spain's record-breaking fourth European Championship title."}

In [17]:
agent_executor.invoke({"input": "Who won the ICC Cricket world cup in 2023?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'ICC Cricket World Cup 2023 winner'}`


[{'url': 'https://en.wikipedia.org/wiki/2023_Cricket_World_Cup_final', 'content': 'The 2023 Cricket World Cup Final was a One Day International cricket match played at the Narendra Modi Stadium in Ahmedabad, India, on 19 November 2023 to determine the winner of the 2023 Cricket World Cup.[2] It was played between host nation India and Australia.[3] It was the first time that Ahmedabad hosted a Cricket World Cup final.[4] It was the second time that India and Australia played a World Cup final against each other, after the 2003 final.[5] [...] Jump to content\nMain menu\nSearch\nDonate\nCreate account\nLog in\nPersonal tools\nToggle the table of contents\n2023 Cricket World Cup final\n4 languages\nArticle\nTalk\nRead\nEdit\nView history\nTools\nFrom Wikipedia, the free encyclopedia\n2023 Cricket World Cup Final\nMitchell Starc bowling against Rohit Shar

{'input': 'Who won the ICC Cricket world cup in 2023?',
 'output': 'Australia won the ICC Cricket World Cup in 2023 by defeating India in the final. The final match was played on 19 November 2023 at the Narendra Modi Stadium in Ahmedabad, India. Australia emerged victorious by 6 wickets.'}

In [19]:
import os
from typing import List, Optional

from langchain.agents import AgentType, initialize_agent
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms import OpenAI
from langchain.schema import AgentAction, AgentFinish, LLMResult
from langchain.tools import BaseTool
from tavily import TavilyClient


In [20]:
# Set your Tavily API key as an environment variable
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"  # Replace with your actual API key


In [21]:
class TavilySearchTool(BaseTool):
    name = "Tavily Search"
    description = "Useful for when you need to answer questions about current events. You should ask targeted questions."

    def _run(self, query: str, run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:
        """Use the Tavily API to search for the query."""
        tavily = TavilyClient()
        results = tavily.search(query)
        return str(results)

    async def _arun(self, query: str, run_manager: Optional[CallbackManagerForLLMRun] = None) -> str:
        """Use the Tavily API to search for the query."""
        tavily = TavilyClient()
        results = tavily.search(query)
        return str(results)

In [22]:

# Initialize the language model
llm = OpenAI(temperature=0)  # You can adjust the temperature

# Initialize the Tavily tool
tavily_tool = TavilySearchTool()

In [23]:
# Create a list of tools for the agent to use
tools = [tavily_tool]

In [33]:

# # Initialize the agent
# agent = initialize_agent(
#     tools,
#     # llm,
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #or other agent types
#     verbose=True, #set to true to see the agents thought process.
# )
agent = create_tool_calling_agent(model, tools, prompt)


In [35]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, stream_runnable=False)

In [36]:
# Run the agent with a query
query = "What are the latest news about the Mars rover?"
# response = agent.run(query)

# print(response)

agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest news about Mars rover'}`


[{'url': 'https://www.bbc.com/news/topics/cqj8yrnmjjmt', 'content': "Mars life clues deposited ready for Earth return. Nasa's Perseverance Mars rover begins building a store of rocks for retrieval by a later mission. 22 Dec 2022."}, {'url': 'https://www.sciencedaily.com/news/space_time/mars/', 'content': "Oct. 7, 2024 — NASA's Curiosity rover, currently exploring Gale crater on Mars, is providing new details about how the ancient Martian climate went from"}, {'url': 'https://www.the-independent.com/topic/curiosity-rover', 'content': "Nasa rover has found previously undetected organic molecules on Mars\n\nEvidence of ancient life on Mars has been ‘erased’, Nasa says\n\nNext stop Mars: 3 spacecraft arriving in quick succession\n\nNasa gets inexplicable new data showing oxygen fluctuations on Mars\n\nNasa Mars rover discovers gas that suggests recent alien lif

{'input': 'What are the latest news about the Mars rover?',
 'output': 'Here are some of the latest news about the Mars rover:\n1. [BBC News: "Mars life clues deposited ready for Earth return"](https://www.bbc.com/news/topics/cqj8yrnmjjmt) - Nasa\'s Perseverance Mars rover begins building a store of rocks for retrieval by a later mission. (Published on 22 Dec 2022)\n2. [Science Daily: NASA\'s Curiosity rover providing new details about ancient Martian climate](https://www.sciencedaily.com/news/space_time/mars/) - NASA\'s Curiosity rover, currently exploring Gale crater on Mars, is providing new details about how the ancient Martian climate evolved.\n3. [The Independent: Various news updates about the Curiosity rover](https://www.the-independent.com/topic/curiosity-rover) - Nasa rover has found previously undetected organic molecules on Mars, evidence of ancient life on Mars has been ‘erased’, recent discoveries of gas that suggests recent alien life, and more.'}

In [37]:
query = "What are the recent advancements in AI?"
# response = agent.run(query)
agent_executor.invoke({"input":query})





> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'recent advancements in AI'}`


[{'url': 'https://indatalabs.com/blog/ai-latest-developments', 'content': 'Source: Unsplash\nSome of the most recent advancements in this field include the release of Magnifier, a behavioral analytics solution from Palo Alto Networks. The latest AI developments solution models network behavior by using machine learning to enhance threat detection.\nAlphabet, the parent company of Google, has introduced the cybersecurity intelligence platform Chronicle to streamline and improve cybersecurity measures as we know them.\n#9: AI applications in fintech [...] Source: Unsplash\nIt’s difficult for machines to identify rare diseases due to image shortages. Recent developments in AI have allowed for the augmentation of AI training by computer-generated X-rays to create a larger database. Through this database, neural networks are trained to identify rare conditions.\n#4

{'input': 'What are the recent advancements in AI?',
 'output': "Here are some recent advancements in AI:\n\n1. The release of Magnifier, a behavioral analytics solution from Palo Alto Networks, which models network behavior using machine learning to enhance threat detection.\n2. Alphabet (Google's parent company) introduced the cybersecurity intelligence platform Chronicle to streamline cybersecurity measures.\n3. Advancements in AI have enabled the augmentation of AI training by using computer-generated X-rays to create a larger database for identifying rare diseases.\n4. Advancements in AI applications in the automobile sector.\n5. Evolution of AI trends in robotics, healthcare, cybersecurity, fintech, and smartphone apps.\n6. Advancements in AutoML for automating machine learning processes, Multi-modal AI combining text, image, and speech processing, Explainable AI for transparency, enhanced Computer Vision capabilities, and Neural Networks for more efficient learning.\n7. Meta int

In [39]:
# print(response)

query = "Who is the current CEO of Microsoft?"
# response = agent.run(query)

# print(response)
agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current CEO of Microsoft'}`


[{'url': 'https://en.wikipedia.org/wiki/Satya_Nadella', 'content': 'Satya Narayana Nadella (/nəˈdɛlə/; born 19 August 1967) is an Indian-born American business executive who is the chairman and chief executive officer (CEO) of Microsoft, succeeding Steve Ballmer in 2014 as CEO[2][3] and John W. Thompson in 2021 as chairman.[4][5] Before becoming CEO, he was the executive vice president of Microsoft\'s cloud and enterprise group, responsible for building and running the company\'s computing platforms.[6]\nEarly life [...] Corporate Vice-president of Business Solutions and Search & Advertising Platform Group\nExecutive Vice-president of Cloud and Enterprise group[34]\nChief executive officer (2014–present)\nOn 4 February 2014, Nadella was announced as the new CEO of Microsoft,[2][35] the third CEO in the company\'s history, following Bill Gates and Steve Ballmer.

{'input': 'Who is the current CEO of Microsoft?',
 'output': 'The current CEO of Microsoft is Satya Nadella. He has been the CEO since February 4, 2014. You can find more information about him on [Wikipedia](https://en.wikipedia.org/wiki/Satya_Nadella).'}